# Build the model for questionanswering transformer from huggingface

In [22]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
import datasets
import torch

In [23]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# Load the dataset
dataset = datasets.load_dataset("squad")


In [36]:
dataset["train"][3]

{'id': '5733be284776f41900661181',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'What is the Grotto at Notre Dame?',
 'answers': {'text': ['a Marian place of prayer and reflection'],
  'answer_start': [381]}}

In [25]:
dataset.shape

{'train': (87599, 5), 'validation': (10570, 5)}

In [26]:
# Create smaller splits for training and validation
train_dataset = dataset["train"].select(list(range(1000)))
valid_dataset = dataset["train"].select(list(range(1000, 1000 + 200)))


In [27]:
# Define the prepare_data function
def prepare_data(example, tokenizer):
    context = example["context"]
    question = example["question"]
    encoding = tokenizer(context, question, truncation=True, padding=True, return_tensors="pt")
    return encoding

# Convert the datasets to a format that the model can understand
encoded_train_dataset = train_dataset.map(lambda example: prepare_data(example, tokenizer), batched=True)
encoded_valid_dataset = valid_dataset.map(lambda example: prepare_data(example, tokenizer), batched=True)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [28]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_steps=500,
    save_total_limit=2,
    logging_steps=50,
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_valid_dataset,
)

# Evaluate the model on the validation dataset
eval_results = trainer.evaluate()

In [29]:
# Print the evaluation results
print(eval_results)

{'eval_runtime': 257.8676, 'eval_samples_per_second': 0.776, 'eval_steps_per_second': 0.05}


In [30]:
# Use the model to answer questions
def answer_question(context, question, tokenizer):
    encoding = tokenizer(context, question, truncation=True, padding=True, return_tensors="pt")
    outputs = model(**encoding)
    
    start_logits = outputs.start_logits.squeeze()
    end_logits = outputs.end_logits.squeeze()

    top_start_index = torch.argmax(start_logits)
    top_end_index = torch.argmax(end_logits) + 1

    # Check if the model did not find a valid answer
    if top_start_index >= top_end_index or top_end_index > len(context):
        return "No answer found"

    answer = context[top_start_index:top_end_index]
    return answer

In [45]:
context = "The quick brown fox jumps over the lazy dog."
question = "What color is the fox?"

answer = answer_question(context, question, tokenizer)
print(answer)

T


## Output
 I trained the model with small dataset. So it provieded the incorrect output.To improve the accuracy of the model, we could try using a larger dataset that contains more examples of questions and answers related to colors of animals.